In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer\\research'

In [3]:
os.chdir("../") 

In [4]:
%pwd


'd:\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir:Path
    data_path : Path
    tokenizer_name : Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import create_directories

[2025-11-17 17:08:35,421: INFO: __init__: Logging system is working correctly!]


In [7]:

from TextSummarizer.utils.common import read_yaml


class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        # Load YAML
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create root folder
        create_directories([Path(self.config.artifacts_root)])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        # Create directory
        create_directories([config.root_dir])

        # Prepare dataclass object
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config


In [8]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


C:\Users\insph\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
from transformers import AutoTokenizer
from datasets import load_from_disk


class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # Encode input dialogue
        input_encodings = self.tokenizer(
            example_batch["dialogue"],
            max_length=1024,
            truncation=True
        )

        # Encode target summary
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch["summary"],
                max_length=128,
                truncation=True
            )

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }

    def convert(self):
        # Load raw dataset
        datasets_samsum = load_from_disk(self.config.data_path)

        # Apply tokenization
        datasets_samsum_pt = datasets_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )

        # Save processed dataset
        output_path = os.path.join(self.config.root_dir, "samsum_dataset")
        datasets_samsum_pt.save_to_disk(output_path)

        return output_path


In [10]:
try:
    config = ConfigurationManager()

    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)

    data_transformation.convert()

except Exception as e:
    logger.exception(e)

[2025-11-17 17:08:45,384: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2025-11-17 17:08:45,386: INFO: common: YAML file 'params.yaml' loaded successfully]
[2025-11-17 17:08:45,388: INFO: common: Created directory at: artifacts]
[2025-11-17 17:08:45,389: INFO: common: Created directory at: artifacts\data_transformation]


C:\Users\insph\AppData\Roaming\Python\Python314\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\insph\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]C:\Users\insph\AppData\Roaming\Python\Pytho